# Welcome to the ALI Troubleshooting Runbook! - WEI VERSION

Please use the cells below to run the queries mentioned in the ALI Troubleshooting Runbook: https://corpwiki.appnexus.com/x/7pC1Bg

** To run each cell, press SHIFT + ENTER **

If you are making changes to the queries for your own analytical purposes, please ** make a copy ** of the notebook before doing so.

Reach out to Austin Greenfield (agreenfield@appnexus.com) with any questions or suggestions for this runbook!

In [5]:
### Use this cell to establish connections to the databases 
# This is the first cell you will run! 
# To run this cell, use SHIFT + ENTER
import pandas as pd
from link import lnk
import cia_library as cia
import os

import sys
sys.path.insert(0, '/home/ezoidis/ipynb/cia_atg-product/ali_release')
from release_utils import *
from queries import *

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 1000)

*** Please enter your ALI ID below. This will be automatically populated in the queries to follow. ***

In [6]:
## Enter your ALI ID below
line_item_id = 4095216

** STEP 2: Analyze Delivery and Changes **

In [7]:
# 2.2 Pull Transaction Logs
#-------------------------------------------------------------------------------------------------------
transaction = lnk.dbs.prod.changelog

## Define start and end dates for change logs
start_date = '2017-05-09'
end_date = '2017-05-13'

translog_query = """SELECT 
    td.created_on as change_time,
    td.field as changed_setting,
    td.old as old_setting,
    td.new as new_setting,
    t.method,td.id,
    td.transaction_id,
    t.service,
    td.resource_id as object_id,
    t.admin_id,td.user_id 
FROM 
    transaction.transaction_detail td 
LEFT JOIN 
    transaction.transaction t 
ON 
    td.transaction_id=t.id
WHERE 
    t.resource_id IN ({ali})
    AND td.user_id != 3186
    AND td.created_on >='{start_date}'
    AND td.created_on <= '{end_date}'
ORDER BY 1""".format(ali=line_item_id,start_date=start_date,end_date=end_date)
translog_df = transaction.select_dataframe(translog_query)
translog_df

,change_time,changed_setting,old_setting,new_setting,method,id,transaction_id,service,object_id,admin_id,user_id
0,2017-05-10 06:10:44,None,None,"{""id"":4095216,""code"":null,""name"":""Popular123_test for SimilarWeb"",""advertiser_id"":1609099,""state"":""active"",""start_date"":null,""end_date"":null,""timezone"":""Asia\/Hong_Kong"",""discrepancy_pct"":0,""publishers_allowed"":""all"",""revenue_value"":null,""revenue_type"":""vcpm"",""goal_type"":""cpc"",""goal_value"":null,""last_modified"":""2017-05-10 06:10:43"",""click_url"":""www.popular123.com"",""currency"":""USD"",""require_cookie_for_tracking"":true,""profile_id"":83239779,""member_id"":7459,""flat_fee_type"":null,""comments"":null,""remaining_days"":null,""total_days"":null,""manage_creative"":true,""creative_distribution_type"":""even"",""line_item_type"":""standard_v2"",""prefer_delivery_over_performance"":true,""advertiser"":{""id"":1609099,""name"":""Westwin In-House Ads_websites""},""flat_fee"":null,""delivery_goal"":null,""labels"":null,""broker_fees"":null,""pixels"":null,""insertion_orders"":[{""id"":498116,""state"":""active"",""code"":null,""name"":""IO_Popular123.com"",""advertiser_id"":1609099,""start_date"":null,""end_date"":null,""timezone"":""Asia\/Hong_Kong"",""las...",post,3397228229,25837b75-788b-5e7f-9688-10c61ea939fb,line-item,4095216,None,200902
1,2017-05-10 06:12:03,budget_intervals,"[{""id"":1688878,""object_id"":4095216,""object_type"":""campaign_group"",""start_date"":""2017-04-20 00:00:00"",""end_date"":""2017-05-01 00:00:00"",""timezone"":""Asia\/Hong_Kong"",""parent_interval_id"":1466551,""lifetime_budget"":0,""lifetime_budget_imps"":null,""lifetime_pacing"":true,""enable_pacing"":true,""lifetime_pacing_pct"":100}]","[{""parent_interval_id"":1688881,""start_date"":""2017-05-10 00:00:00"",""end_date"":""2017-05-12 00:00:00"",""lifetime_budget"":50,""lifetime_budget_imps"":null,""lifetime_pacing"":true,""enable_pacing"":true,""timezone"":""Asia\/Hong_Kong""},{""id"":1688878,""object_id"":4095216,""object_type"":""campaign_group"",""start_date"":""2017-04-20 00:00:00"",""end_date"":""2017-05-01 00:00:00"",""timezone"":""Asia\/Hong_Kong"",""parent_interval_id"":1466551,""lifetime_budget"":0,""lifetime_budget_imps"":null,""lifetime_pacing"":true,""enable_pacing"":true,""lifetime_pacing_pct"":100}]",put,3397231181,498a4119-a226-507c-b72b-0363d10fd108,line-item,4095216,None,200902
2,2017-05-10 06:12:03,valuation,"{""min_margin_pct"":null,""max_avg_cpm"":1,""min_avg_cpm"":0.6,""goal_target"":0.2,""goal_threshold"":0.2,""no_revenue_log"":false,""performance_offer"":false,""performance_mkt_managed"":false,""performance_mkt_crossnet"":false,""bid_price_pacing_enabled"":false,""bid_price_pacing_lever"":0,""campaign_group_valuation_strategy"":null,""goal_confidence_threshold"":null}","{""performance_mkt_managed"":false}",put,3397231187,498a4119-a226-507c-b72b-0363d10fd108,line-item,4095216,None,200902
3,2017-05-10 06:12:03,budget_allocation_re,None,1,put,3397231189,498a4119-a226-507c-b72b-0363d10fd108,line-item,4095216,None,200902
4,2017-05-10 06:17:52,valuation,"{""min_margin_pct"":null,""max_avg_cpm"":1,""min_avg_cpm"":0.6,""goal_target"":0.2,""goal_threshold"":0.2,""no_revenue_log"":false,""performance_offer"":false,""performance_mkt_managed"":false,""performance_mkt_crossnet"":false,""bid_price_pacing_enabled"":false,""bid_price_pacing_lever"":0,""campaign_group_valuation_strategy"":null,""goal_confidence_threshold"":null}","{""campaign_group_valuation_strategy"":null,""goal_target"":0.2,""goal_threshold"":0.2,""max_avg_cpm"":1,""min_avg_cpm"":0.6,""min_margin_pct"":null,""performance_mkt_crossnet"":false,""performance_mkt_managed"":false,""member_id"":""7459""}",put,3397243245,4f53fa18-ea6b-5514-b20d-05506e913536,line-item,4095216,None,200902
5,2017-05-10 06:17:52,inventory_discovery,"{""use_ranked_discovery"":false,""fail_criteria_type"":""booked_revenue"",""fail_criteria_amount"":0.379459}","{""fail_criteria_amount"":0.37945942131684,""fail_c

** STEP 5: Ranked Discovery - Check Available Inventory **

In [4]:
# 1.a.i Pull count of inventory objects moved to discovery states over time
#-------------------------------------------------------------------------------------------------------
spotfire_vert = lnk.dbs.prod.spotfire_vertica

rank_moves_query = """SELECT
    ymdh,
    campaign_group_id,
    COUNT(DISTINCT case when discovery_state_id = 1 THEN inventory_object_id END) as moved_to_test,
    COUNT(DISTINCT case when discovery_state_id = 2 THEN inventory_object_id END) as moved_to_pass,
    COUNT(DISTINCT case when discovery_state_id = 3 THEN inventory_object_id END) as moved_to_fail,
    COUNT(DISTINCT CASE WHEN discovery_state_id = 4 THEN inventory_object_id END) as moved_to_reset
    FROM discovery_history
    WHERE campaign_group_id IN ({ali})
    AND ymdh >= CURRENT_DATE - 2
    GROUP BY 1,2;""".format(ali=line_item_id)
rank_moves_df = spotfire_vert.select_dataframe(rank_moves_query)
rank_moves_df

,ymdh,campaign_group_id,moved_to_test,moved_to_pass,moved_to_fail,moved_to_reset
0,2017-05-09 02:00:00,4055642,5,0,0,0
1,2017-05-09 03:00:00,4055642,6,0,0,0
2,2017-05-09 09:00:00,4055642,6,0,0,0
3,2017-05-09 13:00:00,4055642,6,0,0,0
4,2017-05-09 17:00:00,4055642,6,0,0,0
5,2017-05-10 01:00:00,4055642,6,0,0,0
6,2017-05-08 23:00:00,4055642,6,0,0,0
7,2017-05-09 01:00:00,4055642,6,0,0,0
8,2017-05-09 04:00:00,4055642,6,0,0,0
9,2017-05-09 05:00:00,4055642,6,0,0,0


In [5]:
# 2.b.i Pull the fail criteria amount
#-------------------------------------------------------------------------------------------------------
prod = lnk.dbs.prod.my_api

fail_criteria_query = """SELECT campaign_group_id,
    use_ranked_discovery,
        fail_criteria_type_id,
        fail_criteria_amount
    from bidder.campaign_group_discovery
    where campaign_group_id in ({ali})
        and deleted=0;""".format(ali=line_item_id)
fail_criteria_df = prod.select_dataframe(fail_criteria_query)
fail_criteria_df

,campaign_group_id,use_ranked_discovery,fail_criteria_type_id,fail_criteria_amount
0,4055642,1,2,3.794594


** STEP 6a. Run a Debug Auction (for delivery goal ALIs) **

In [6]:
# 6a.3.a Pull all discovery inventory mappings from spotfire_vertica: Nerdt function - vspotfire
#-------------------------------------------------------------------------------------------------------
spotfire_vert = lnk.dbs.prod.spotfire_vertica

discovery_inventory_query = """SELECT 
        campaign_group_id, 
        inventory_object_id as discovery_inventory_map_id, 
        compute_ts, 
        rank 
    FROM discovery_history 
    WHERE discovery_state_id = 1 
        AND campaign_group_id IN ({ali}) 
    ORDER BY compute_ts DESC;""".format(ali=line_item_id)
discovery_inventory_df = spotfire_vert.select_dataframe(discovery_inventory_query)
discovery_inventory_df

,campaign_group_id,discovery_inventory_map_id,compute_ts,rank
0,4055642,664088,2017-05-10 08:04:33,0.000674
1,4055642,430294,2017-05-10 07:54:33,0.000641
2,4055642,358889,2017-05-10 07:44:32,0.000633
3,4055642,265405,2017-05-10 07:34:32,0.000611
4,4055642,191997,2017-05-10 07:24:32,0.000566
5,4055642,268491,2017-05-10 07:14:32,0.000727
6,4055642,160582,2017-05-10 07:04:31,0.000726
7,4055642,667434,2017-05-10 06:54:30,0.000620
8,4055642,677912,2017-05-10 06:44:28,0.000613
9,4055642,664240,2017-05-10 06:34:26,0.000605


In [7]:
# 6a.3.b Then, pull URL and tag for each inventory mapping from opt_mysql: Nerdt function - mopt
#-------------------------------------------------------------------------------------------------------
opt_mysql = lnk.dbs.prod.opt_mysql

discovery_inventory_map_query = """SELECT 
        id as discovery_inventory_map_id, 
        inventory_url_id,
        tag_id
    FROM 
        optimization.campaign_group_discovery_inventory_map 
    WHERE id IN ({disc_map_id})
    ORDER BY id DESC;""".format(disc_map_id=str(list(discovery_inventory_df['discovery_inventory_map_id'].unique()))[1:-1])
discovery_inventory_map_df = opt_mysql.select_dataframe(discovery_inventory_map_query)
discovery_inventory_map_df

,discovery_inventory_map_id,inventory_url_id,tag_id
0,677937,1535163,9102565
1,677936,860567,4318518
2,677913,769038,10478410
3,677912,771602,10478409
4,677864,1356415,11112085
5,677158,796185,10478409
6,672675,1503375,9884869
7,672606,588309,10929731
8,672589,9546,10478409
9,672588,821383,10478409


** ALI spend by seller, site_domain (originally created for the purpose of troubleshooting ETV ALIs) **

In [64]:
vertica = lnk.dbs.prod.vertica
prod = lnk.dbs.prod.my_api

## Define start and end dates for vertica query, enter your email address to email the results
start_date = '2017-04-01'
end_date = '2017-04-22'
email = 'ezoidis@appnexus.com'

#Pull seller and site domain data from vertica
#Timezone EST
vert_query = """
	SELECT
        a.ymd,
        a.seller_member_id,
        b.seller_member_group_id,
        a.site_domain, 
        sum(a.booked_revenue_dollars) as booked_revenue_dollars,
        sum(a.imps) as imps,
        sum(a.viewed_imps) as viewed_imps,
        sum(a.view_measurable_imps) as view_measurable_imps
	FROM 
        agg_dw_site_domain_perf a
	JOIN 
        sup_common_member_seller_member_group b
	ON 
        a.seller_member_id = b.member_id
	WHERE 
        DATE(a.ymdh) >= DATE('{start_date}') + interval '5 hours'
        AND DATE(a.ymdh) < DATE('{end_date}') + interval '5 hours' 
        AND a.buyer_member_id = 1145
        AND a.campaign_group_id = {ali}
	GROUP BY 1,2,3,4
	ORDER BY 5 DESC;""".format(ali=line_item_id,start_date=start_date,end_date=end_date)
vert_data = vertica.select_dataframe(vert_query)

#Grab seller name from mysql
sql_query = """
	SELECT
        id as seller_member_id,
        billing_name
	FROM 
        api.member
	WHERE 
        deleted = 0
	GROUP BY 1,2;"""
sql_data = prod.select_dataframe(sql_query)

#Merge seller names to vertica data
etv = vert_data.merge(sql_data, how = 'left', on = 'seller_member_id')
filename = 'etv_ali_{ali}.csv'.format(ali=line_item_id)
etv.to_csv(filename, index = False)

## It is now time to email the table
cia.send_email(email, \
               send_from=email, \
               reply_to=email, \
               subject='ETV ALI Data', \
               body='Meow.', \
               attachment_paths=filename)

os.remove(filename)